# Training and Optimizing VGG19

The following goes through the process of fine-tuning a pre-existing VGG19 convolution neural network using the CIFAR10 dataset, then optimizing using pruning

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torchvision.models as models

## Import VGG19 (Pretrained)

In [2]:
model = models.vgg19(pretrained=True)

/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Import and Pre-process Dataset

### Import datasets

In [2]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
train_data = CIFAR10(
    root="./CIFAR10-Train",
    train=True,
    transform=preprocess,
    target_transform=None,
    download=True,
)

100%|██████████| 170M/170M [00:05<00:00, 33.9MB/s] 


In [4]:
test_data = CIFAR10(
    root="./CIFAR10-Test",
    train=False,
    transform=preprocess,
    target_transform=None,
    download=True,
)

100%|██████████| 170M/170M [00:04<00:00, 34.2MB/s] 


### Split dataset and show stats

In [5]:
# Index to split data
split_index = int(len(test_data) / 2)

split_size = len(test_data) // 2
test_set, val_set = random_split(test_data, [split_size, len(test_data) - split_size])

In [6]:
print('------------------------')
print('Stats on CIFA100 Dataset')
print('------------------------')
print(f'Train Dataset Size: {len(train_data)}')
print(f'Train - Number of Unique Targets: {len(train_data.classes)}')
print('')
print(f'Test Dataset Size: {len(test_set)}')
print(f'Test - Number of Unique Targets: {len(test_set.dataset.classes)}')
print('')
print(f'Val Dataset Size: {len(val_set)}')
print(f'Val - Number of Unique Targets: {len(val_set.dataset.classes)}')

------------------------
Stats on CIFA100 Dataset
------------------------
Train Dataset Size: 50000
Train - Number of Unique Targets: 10

Test Dataset Size: 5000
Test - Number of Unique Targets: 10

Val Dataset Size: 5000
Val - Number of Unique Targets: 10


### Pre-process and update model for classifier size

In [7]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=4)

In [17]:
num_classes = len(train_data.classes)

# Convert model's predefined output classifier to new classifier that has our actual number of classes
# Layer 6 is the classifier layer
model.classifier[6] = nn.Linear(4096, num_classes)

In [10]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

## Setting up Model for Training

In [18]:
# We use CrossEntropyLoss for classification loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [12]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
model = model.to(device)

## Defining Helper Functions

### Metric Calculation (F1, Recall, Precision)

In [9]:

def calculate_metrics(classes, df:pd.DataFrame):
    df_copy = df.copy()
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for i in classes:
        true_positives += len(df_copy[(df_copy['y_true'] == i) & (df_copy['y_pred'] == i)])
        false_negatives += len(df_copy[(df_copy['y_true'] == i) & (df_copy['y_pred'] != i)])
        false_positives += len(df_copy[(df_copy['y_true'] != i) & (df_copy['y_pred'] == i)])
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2 * ((precision * recall) / (precision + recall))
    
    return precision, recall, f1

### Train Function

In [14]:
from tqdm import tqdm

def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pred_v_true = pd.DataFrame(columns=['y_true','y_pred'])
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Put back to CPU and put in proper format
        labels_cpu = labels.cpu().numpy()
        predicted_cpu = predicted.cpu().numpy()
        
        batch_df = pd.DataFrame({
            'y_true': labels_cpu, 
            'y_pred': predicted_cpu
        })
        
        pred_v_true = pd.concat([pred_v_true, batch_df], ignore_index=True)
        
    classes_num = range(len(train_data.classes))
    precision, recall, f1 = calculate_metrics(classes_num, pred_v_true)
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc, precision, recall, f1

### Validation Function

In [15]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pred_v_true = pd.DataFrame(columns=['y_true','y_pred'])
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Put back to CPU and put in proper format
            labels_cpu = labels.cpu().numpy()
            predicted_cpu = predicted.cpu().numpy()
            
            batch_df = pd.DataFrame({
                'y_true': labels_cpu, 
                'y_pred': predicted_cpu
            })
            
            pred_v_true = pd.concat([pred_v_true, batch_df], ignore_index=True)
            
    classes_num = range(len(train_data.classes))
    precision, recall, f1 = calculate_metrics(classes_num, pred_v_true)
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc, precision, recall, f1

## Model Training

In [16]:
num_epochs = 10
best_acc = 0.0

# Create lists to store metrics for each epoch
training_history = {
    'epoch': [],
    'train_loss': [],
    'train_acc': [],
    'train_precision': [],
    'train_recall': [],
    'train_f1': [],
    'val_loss': [],
    'val_acc': [],
    'val_precision': [],
    'val_recall': [],
    'val_f1': []
}

# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    # Training phase 
    train_loss, train_acc, train_precision, train_recall, train_f1 = train_epoch(
        model, train_loader, criterion, optimizer, device
    )
    
    # Validation phase 
    val_loss, val_acc, val_precision, val_recall, val_f1 = validate(
        model, val_loader, criterion, device
    )
    
    # Store metrics in history dictionary
    training_history['epoch'].append(epoch + 1)
    training_history['train_loss'].append(train_loss)
    training_history['train_acc'].append(train_acc)
    training_history['train_precision'].append(train_precision)
    training_history['train_recall'].append(train_recall)
    training_history['train_f1'].append(train_f1)
    training_history['val_loss'].append(val_loss)
    training_history['val_acc'].append(val_acc)
    training_history['val_precision'].append(val_precision)
    training_history['val_recall'].append(val_recall)
    training_history['val_f1'].append(val_f1)
    
    # Print training metrics
    print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.2f}%')
    print(f'Train Precision: {train_precision:.4f} Recall: {train_recall:.4f} F1: {train_f1:.4f}')
    
    # Print validation metrics
    print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.2f}%')
    print(f'Val Precision: {val_precision:.4f} Recall: {val_recall:.4f} F1: {val_f1:.4f}')
    
    # Update best accuracy if current validation accuracy is better
    if val_acc > best_acc:
        best_acc = val_acc
        print(f'New best validation accuracy: {best_acc:.2f}%')
    
    print()

# Print final best accuracy
print(f'Training completed - Best validation accuracy: {best_acc:.2f}%')

# Create DataFrame from training history
training_df = pd.DataFrame(training_history)

# Export to CSV file
training_df.to_csv('./training_history_CIFAR10.csv', index=False)
torch.save(model.state_dict(), './CIFAR10_VGG19_Model.pth')

Epoch 1/10
----------


  0%|          | 0/1563 [00:00<?, ?it/s]

100%|██████████| 157/157 [01:10<00:00,  2.23it/s]


Train Loss: 0.4924 Acc: 83.36%
Train Precision: 0.8336 Recall: 0.8336 F1: 0.8336
Val Loss: 0.3136 Acc: 89.36%
Val Precision: 0.8936 Recall: 0.8936 F1: 0.8936
New best validation accuracy: 89.36%

Epoch 2/10
----------


100%|██████████| 157/157 [01:08<00:00,  2.30it/s]


...

Train Loss: 0.0770 Acc: 97.52%
Train Precision: 0.9752 Recall: 0.9752 F1: 0.9752
Val Loss: 0.3301 Acc: 91.38%
Val Precision: 0.9138 Recall: 0.9138 F1: 0.9138

Epoch 9/10
----------


100%|██████████| 157/157 [01:07<00:00,  2.32it/s]


Train Loss: 0.0695 Acc: 97.77%
Train Precision: 0.9777 Recall: 0.9777 F1: 0.9777
Val Loss: 0.4218 Acc: 89.64%
Val Precision: 0.8964 Recall: 0.8964 F1: 0.8964

Epoch 10/10
----------


100%|██████████| 157/157 [01:07<00:00,  2.32it/s]


Train Loss: 0.0657 Acc: 97.90%
Train Precision: 0.9790 Recall: 0.9790 F1: 0.9790
Val Loss: 0.3883 Acc: 90.92%
Val Precision: 0.9092 Recall: 0.9092 F1: 0.9092

Training completed - Best validation accuracy: 92.08%


## Testing

### Import Model

In [88]:
# Recreate the model architecture
model = models.vgg19(pretrained=True)

# Modify final layer if needed for your classes
model.classifier[6] = torch.nn.Linear(4096, num_classes)

# Load the state dict
model.load_state_dict(torch.load('./VGG19_Models/Trained_Model/CIFAR10_VGG19_Model.pth'))
model.eval()

/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

### Testing (No Pruning)

In [79]:
import time

pred_v_true = pd.DataFrame(columns=['y_true','y_pred'])
running_loss = 0.0
correct = 0
total = 0

testing_metrics = {
    'test_loss': [],
    'test_acc': [],
    'test_precision': [],
    'test_recall': [],
    'test_f1': [],
    'latency': [],
    'throughput': [],
}

latency_times = []
loop_start = time.perf_counter()
with torch.no_grad():
    for inputs, labels in tqdm(test_set):
        
        # Convert Inputs and Outputs to proper format for model
        inputs = inputs.unsqueeze(0)
        labels = torch.tensor([labels])
        
        # Get model prediction and total inference time
        inference_start = time.perf_counter()
        outputs = model(inputs)
        inference_end = time.perf_counter()
        latency_times.append(inference_end - inference_start)
        
        loss = criterion(outputs, labels)
        
        # Tally Model Metrics - Loss, Prediction vs Correct values
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        batch_df = pd.DataFrame({
            'y_true': labels, 
            'y_pred': predicted
        })
        
        pred_v_true = pd.concat([pred_v_true, batch_df], ignore_index=True)

    loop_end = time.perf_counter()
    
# Calculate Hardware Metrics - Latency and Throughput
avg_latency = np.mean(latency_times)
throughput = total / (loop_end - loop_start)

# Calculate Test Precision, Recall, F1
classes_num = range(1, len(train_data.classes) + 1)
test_precision, test_recall, test_f1 = calculate_metrics(classes_num, pred_v_true)

# Calculate Basic Model Metrics  - Loss and Accuracy
test_loss = running_loss / len(val_loader)
test_acc = 100. * correct / total

# Add Metrics to Dictionary
testing_metrics['test_loss'].append(test_loss)
testing_metrics['test_acc'].append(test_acc)
testing_metrics['test_precision'].append(test_precision)
testing_metrics['test_recall'].append(test_recall)
testing_metrics['test_f1'].append(test_f1)
testing_metrics['latency'].append(avg_latency)
testing_metrics['throughput'].append(throughput)
testing_metrics = pd.DataFrame(testing_metrics)

# Print testing metrics
print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.2f}%')
print(f'Test Precision: {test_precision:.4f} Recall: {test_recall:.4f} F1: {test_f1:.4f}')
testing_metrics.to_csv('./test_history_NO_PRUNE.csv', index=False)

100%|██████████| 5000/5000 [06:03<00:00, 13.75it/s]

Test Loss: 12.5724 Acc: 90.42%
Test Precision: 0.9009 Recall: 0.9015 F1: 0.9012


### Testing (With Structured Pruning)

#### Pruning

Pruning is a way to remove "insignificant" weights in a trained model. In return, this leads to quicker inference times due to less computation (fewer MAC operations).

The pruning method used will be an unstructured approach. The unstructured approach removes single weigths of filters from a convolutional neural network, which VGG19 utilizes. This removal of single weights allows for easier model acceleration on hardware (compared to unstructured pruning, which creates irregular sparsity patterns), which is our reason for pruning.

Earlier layers in VGG19 are more sensitive to pruning, as they capture fundamental low-level features like edges, textures, and basic shapes. Later layers are less sensitive as they learn more abstract, high-level features that often have more redundancy. So we will use a low percentage of pruning for the first couple layers and increase the percentage as we proceed downstream in the VGG19 model.

In [10]:
import torch.nn.utils.prune as prune
def prune_model(model, initial_prune_percent, max_prune_percent, increment_percent, percent_rate_change):
    i = 0
    for feat in model.features:
        if str(feat).startswith('Conv2d'):
            i += 1 # Tracks which conv2d layer we are at
            if(i % percent_rate_change == 0):
                if initial_prune_percent < max_prune_percent:
                    initial_prune_percent +=  increment_percent
                initial_prune_percent = round(initial_prune_percent, 2)
            
            # Structured Pruning based on L1 normalization
            prune.l1_unstructured(
                feat,
                name="weight",
                amount=initial_prune_percent,
            )
            prune.remove(feat, "weight")
    

In [11]:
low_pruning = {
    'setting_type': 'low',
    'init_perc': .05,       # Initial Pruning Percentage
    'max_perc': .5,         # Max Pruning Percentage
    'increment_perc': .05,  # How much to increase pruning by
    'perc_rate': 3          # How often to increase pruning by
}

medium_pruning = {
    'setting_type': 'medium',
    'init_perc': .1,
    'max_perc': .7,
    'increment_perc': .15,
    'perc_rate': 3
}

high_pruning = {
    'setting_type': 'high',
    'init_perc': .2,
    'max_perc': .8,
    'increment_perc': .15,
    'perc_rate': 3
}

prune_settings = [low_pruning, medium_pruning, high_pruning]

In [19]:
import time
from tqdm import tqdm
for setting in prune_settings:
    # Recreate the model architecture
    model = models.vgg19(pretrained=True)

    # Modify final layer if needed for your classes
    model.classifier[6] = torch.nn.Linear(4096, num_classes)

    # Load the state dict
    model.load_state_dict(torch.load('../VGG19_Models/Trained_Model/CIFAR10_VGG19_Model.pth'))
    model.eval()

    prune_model(
        model, 
        setting['init_perc'], 
        setting['max_perc'], 
        setting['increment_perc'],
        setting['perc_rate']
    )
    
    pred_v_true = pd.DataFrame(columns=['y_true','y_pred'])
    running_loss = 0.0
    correct = 0
    total = 0

    testing_metrics = {
        'test_loss': [],
        'test_acc': [],
        'test_precision': [],
        'test_recall': [],
        'test_f1': [],
        'latency': [],
        'throughput': [],
        'inital_prune':[],
        'max_prune_perc':[],
        'increment_by': [],
        'rate_of_increase':[]
    }

    latency_times = []
    loop_start = time.perf_counter()
    with torch.no_grad():
        for inputs, labels in tqdm(test_set):
            
            # Convert Inputs and Outputs to proper format for model
            inputs = inputs.unsqueeze(0)
            labels = torch.tensor([labels])
            
            # Get model prediction and total inference time
            inference_start = time.perf_counter()
            outputs = model(inputs)
            inference_end = time.perf_counter()
            latency_times.append(inference_end - inference_start)
            
            loss = criterion(outputs, labels)
            
            # Tally Model Metrics - Loss, Prediction vs Correct values
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            batch_df = pd.DataFrame({
                'y_true': labels, 
                'y_pred': predicted
            })
            
            pred_v_true = pd.concat([pred_v_true, batch_df], ignore_index=True)

        loop_end = time.perf_counter()
        
    # Calculate Hardware Metrics - Latency and Throughput
    avg_latency = np.mean(latency_times)
    throughput = total / (loop_end - loop_start)

    # Calculate Test Precision, Recall, F1
    classes_num = range(1, len(train_data.classes) + 1)
    test_precision, test_recall, test_f1 = calculate_metrics(classes_num, pred_v_true)

    # Calculate Basic Model Metrics  - Loss and Accuracy
    test_loss = running_loss / len(val_loader)
    test_acc = 100. * correct / total

    # Add Metrics to Dictionary
    testing_metrics['test_loss'].append(test_loss)
    testing_metrics['test_acc'].append(test_acc)
    testing_metrics['test_precision'].append(test_precision)
    testing_metrics['test_recall'].append(test_recall)
    testing_metrics['test_f1'].append(test_f1)
    testing_metrics['latency'].append(avg_latency)
    testing_metrics['throughput'].append(throughput)
    testing_metrics['inital_prune'].append(setting['init_perc'])
    testing_metrics['max_prune_perc'].append(setting['max_perc'])
    testing_metrics['increment_by'].append(setting['increment_perc'])
    testing_metrics['rate_of_increase'].append(setting['perc_rate'])
    testing_metrics = pd.DataFrame(testing_metrics)

    # Print testing metrics
    print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.2f}%')
    print(f'Test Precision: {test_precision:.4f} Recall: {test_recall:.4f} F1: {test_f1:.4f}')
    string = f"./test_history_unstructured_prune_{setting['setting_type']}_setting.csv"
    testing_metrics.to_csv(string, index=False)

/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 5000/5000 [05:58<00:00, 13.93it/s]
/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/module1-ece438/lib/

Test Loss: 12.4355 Acc: 90.26%
Test Precision: 0.9002 Recall: 0.8992 F1: 0.8997


100%|██████████| 5000/5000 [06:05<00:00, 13.67it/s]
/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/module1-ece438/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Loss: 17.2473 Acc: 86.36%
Test Precision: 0.8571 Recall: 0.8601 F1: 0.8586


100%|██████████| 5000/5000 [05:46<00:00, 14.43it/s]

Test Loss: 38.8586 Acc: 73.88%
Test Precision: 0.7207 Recall: 0.7324 F1: 0.7265


In [89]:

i = 0                
initial_perc = .1    
max_perc = .7        
increment_perc = .1  
perc_adjust_rate = 3 

for feat in model.features:
    if str(feat).startswith('Conv2d'):
        i += 1 # Tracks which conv2d layer we are at
        if(i % perc_adjust_rate == 0):
            if initial_perc < max_perc:
                initial_perc +=  increment_perc
            initial_perc = round(initial_perc, 2)
        
        # Structured Pruning based on L1 normalization
        prune.l1_unstructured(
            feat,
            name="weight",
            amount=initial_perc,
        )
        prune.remove(feat, "weight")
    

#### Testing (With Structured Pruning)

In [90]:
import time

pred_v_true = pd.DataFrame(columns=['y_true','y_pred'])
running_loss = 0.0
correct = 0
total = 0

testing_metrics = {
    'test_loss': [],
    'test_acc': [],
    'test_precision': [],
    'test_recall': [],
    'test_f1': [],
    'latency': [],
    'throughput': [],
}

latency_times = []
loop_start = time.perf_counter()
with torch.no_grad():
    for inputs, labels in tqdm(test_set):
        
        # Convert Inputs and Outputs to proper format for model
        inputs = inputs.unsqueeze(0)
        labels = torch.tensor([labels])
        
        # Get model prediction and total inference time
        inference_start = time.perf_counter()
        outputs = model(inputs)
        inference_end = time.perf_counter()
        latency_times.append(inference_end - inference_start)
        
        loss = criterion(outputs, labels)
        
        # Tally Model Metrics - Loss, Prediction vs Correct values
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        batch_df = pd.DataFrame({
            'y_true': labels, 
            'y_pred': predicted
        })
        
        pred_v_true = pd.concat([pred_v_true, batch_df], ignore_index=True)

    loop_end = time.perf_counter()
    
# Calculate Hardware Metrics - Latency and Throughput
avg_latency = np.mean(latency_times)
throughput = total / (loop_end - loop_start)

# Calculate Test Precision, Recall, F1
classes_num = range(1, len(train_data.classes) + 1)
test_precision, test_recall, test_f1 = calculate_metrics(classes_num, pred_v_true)

# Calculate Basic Model Metrics  - Loss and Accuracy
test_loss = running_loss / len(val_loader)
test_acc = 100. * correct / total

# Add Metrics to Dictionary
testing_metrics['test_loss'].append(test_loss)
testing_metrics['test_acc'].append(test_acc)
testing_metrics['test_precision'].append(test_precision)
testing_metrics['test_recall'].append(test_recall)
testing_metrics['test_f1'].append(test_f1)
testing_metrics['latency'].append(avg_latency)
testing_metrics['throughput'].append(throughput)
testing_metrics = pd.DataFrame(testing_metrics)

# Print testing metrics
print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.2f}%')
print(f'Test Precision: {test_precision:.4f} Recall: {test_recall:.4f} F1: {test_f1:.4f}')
testing_metrics.to_csv('./test_history_unstructured_PRUNE_High_Settings.csv', index=False)

100%|██████████| 5000/5000 [05:52<00:00, 14.17it/s]

Test Loss: 148.5001 Acc: 30.72%
Test Precision: 0.2879 Recall: 0.3102 F1: 0.2987


**Low Settings**
- i = 0               
- initial_perc = .1   
- max_perc = .7        
- increment_perc = .1
- perc_adjust_rate = 3 

**High Setting**
- i = 0               
- initial_perc = .2   
- max_perc = .7        
- increment_perc = .2
- perc_adjust_rate = 2